In [138]:
import time
import numpy as np
import random
import re
from typing import List, Tuple
import pandas as pd

In [139]:
# naive deck generation

def gen_function(seed, num_run):
    rcards = ['1'] * 26 
    bcards = ['0'] * 26
    cards = rcards+bcards # total 52 cards
    total = []

    for gamenum in range(num_run):
        random.seed(seed)
        random.shuffle(cards)
        #print(cards)
        total.append(''.join(cards))
    return total

In [140]:
def game_function(deck, p1_seq, p2_seq):
    outcomes = []
    stack = [] # to keep track of the cards
    p1_num_tricks = 0
    p2_num_tricks = 0
    p1_num_cards = 0
    p2_num_cards = 0
    
    for card in deck:
        stack.append(card)
        curstack = ''.join(stack[-3:])
        if curstack == p1_seq:
            p1_num_tricks += 1
            p1_num_cards += len(stack)
            stack = []
        elif curstack == p2_seq:
            p2_num_tricks += 1
            p2_num_cards += len(stack)
            stack = []
    return p1_num_tricks, p2_num_tricks, p1_num_cards, p2_num_cards

In [141]:
# base pseudocode
def whole_simulation(seed, num_run):
    i = 0
    game_df = pd.DataFrame(columns = ['seed', 'deck', 'p1_seq', 'p2_seq', 'p1_num_tricks', 'p2_num_tricks', 'p1_num_cards', 'p2_num_cards'])
    decks = gen_function(seed, num_run)
    for deck in decks:
        for p1_seq in ['000', '001', '010', '011', '100', '101', '110', '111']:
            for p2_seq in ['000', '001', '010', '011', '100', '101', '110', '111']:
                p1_num_tricks, p2_num_tricks, p1_num_cards, p2_num_cards = game_function(deck, p1_seq, p2_seq)
                game_df.loc[i] = [seed, deck, p1_seq, p2_seq, p1_num_tricks, p2_num_tricks, p1_num_cards, p2_num_cards]
                i += 1
    return game_df

In [142]:
import random
import numpy as np
import sqlite3
import pandas as pd

In [143]:
import penney_db1
from importlib import reload
reload(penney_db1);
from  penney_db1 import DB

In [144]:
simulation = whole_simulation(1, 10)

In [145]:
simulation.head()

,seed,deck,p1_seq,p2_seq,p1_num_tricks,p2_num_tricks,p1_num_cards,p2_num_cards
0,1,0101101101000110010010011001011110100101110000...,000,000,2,0,45,0
1,1,0101101101000110010010011001011110100101110000...,000,001,2,6,20,32
2,1,0101101101000110010010011001011110100101110000...,000,010,1,7,6,39
3,1,0101101101000110010010011001011110100101110000...,000,011,2,6,9,39
4,1,0101101101000110010010011001011110100101110000...,000,100,0,8,0,51


In [146]:
db = DB()

In [147]:
db.connect_db()

In [148]:
db.insert_results(simulation)

In [149]:
sql ='''
SELECT * FROM results
;'''
db.run_query(sql)

,seed,deck,p1_seq,p2_seq,p1_num_tricks,p2_num_tricks,p1_num_cards,p2_num_cards
0,1,0101101101000110010010011001011110100101110000...,000,000,2,0,45,0
1,1,0101101101000110010010011001011110100101110000...,000,001,2,6,20,32
2,1,0101101101000110010010011001011110100101110000...,000,010,1,7,6,39
3,1,0101101101000110010010011001011110100101110000...,000,011,2,6,9,39
4,1,0101101101000110010010011001011110100101110000...,000,100,0,8,0,51
...,...,...,...,...,...,...,...,...
635,1,0011111000010001110010011110010100001101101101...,111,011,1,7,3,44
636,1,0011111000010001110010011110010100001101101101...,111,100,3,6,12,39
637,1,0011111000010001110010011110010100001101101101...,111,101,3,5,26,23
638,1,0011111000010001110010011110010100001101101101...,111,110,3,5,23,25


In [151]:
sql1 ='''
CREATE VIEW win_results AS
SELECT 
    p1_seq,
    p2_seq,
    p1_num_cards,
    p2_num_cards,
    p1_num_tricks,
    p2_num_tricks,
    deck,
    CASE 
        WHEN p1_num_cards > p2_num_cards THEN 'p1'
        WHEN p1_num_cards < p2_num_cards THEN 'p2'
        ELSE 'draw'
    END AS win_cards,
    CASE 
        WHEN p1_num_tricks > p2_num_tricks THEN 'p1'
        WHEN p1_num_tricks < p2_num_tricks THEN 'p2'
        ELSE 'draw'
    END AS win_tricks
FROM 
    results;
'''
db.run_action(sql1)

In [160]:
sql2 =''' 
select distinct(deck) from win_results;'''
db.run_query(sql2)

,deck
0,0101101101000110010010011001011110100101110000...
1,0110000000110011101100110010010110010101111101...
2,1011101110101111001100100101001100001110001001...
3,1011111000000100100010001001011100111011100101...
4,0000110011111001001101010010010000110011110101...
5,1100011111101110111111000111001000000010001000...
6,1100011001010000010010011101001110111101100100...
7,1110000111011011101100111010000010110101010101...
8,0111101110001110000111100110001100100110011000...
9,0011111000010001110010011110010100001101101101...


In [158]:
sql1 = """
SELECT 
    all_combinations.p1_seq, 
    all_combinations.p2_seq, 
    COALESCE(COUNT(win_results.win_cards), 0) AS p1_wins_by_cards,
    (SELECT COUNT(DISTINCT deck) FROM win_results) AS total_distinct_decks
FROM 
    (SELECT DISTINCT p1_seq, p2_seq FROM win_results) AS all_combinations
LEFT JOIN 
    win_results ON all_combinations.p1_seq = win_results.p1_seq 
                AND all_combinations.p2_seq = win_results.p2_seq
                AND win_results.win_cards = 'p1'
GROUP BY 
    all_combinations.p1_seq, 
    all_combinations.p2_seq
"""
db.run_query(sql1)


,p1_seq,p2_seq,p1_wins_by_cards,total_distinct_decks
0,000,000,10,10
1,000,001,3,10
2,000,010,2,10
3,000,011,2,10
4,000,100,0,10
...,...,...,...,...
59,111,011,0,10
60,111,100,1,10
61,111,101,4,10
62,111,110,3,10


In [161]:
sql1 = """
SELECT 
    all_combinations.p1_seq, 
    all_combinations.p2_seq, 
    COALESCE(COUNT(win_results.win_tricks), 0) AS p1_wins_by_tricks,
    (SELECT COUNT(DISTINCT deck) FROM win_results) AS total_distinct_decks 
FROM 
    (SELECT DISTINCT p1_seq, p2_seq FROM win_results) AS all_combinations
LEFT JOIN 
    win_results ON all_combinations.p1_seq = win_results.p1_seq 
                AND all_combinations.p2_seq = win_results.p2_seq
                AND win_results.win_tricks = 'p1'
GROUP BY 
    all_combinations.p1_seq, 
    all_combinations.p2_seq
"""
db.run_query(sql1)

,p1_seq,p2_seq,p1_wins_by_tricks,total_distinct_decks
0,000,000,10,10
1,000,001,3,10
2,000,010,2,10
3,000,011,1,10
4,000,100,0,10
...,...,...,...,...
59,111,011,0,10
60,111,100,2,10
61,111,101,2,10
62,111,110,4,10


In [ ]:
##trying now to see the probability of player 1 winning by card amount for each different combination
## I used a left join so 64 rows would be returned (some iterations p1 didnt win by card at all for the 3 decks
##the left join made it so that those rows w/o any p1 listed for wincard were null --> used IFNULL(
##or if P1WinCardCount was 0 when it was 0/3 it orginally returned null so I also made it 0
sql1="""
SELECT p1_seq, p2_seq, IFNULL(p1_wins_by_cardss,0) AS P1CardWinCount , TimesPlayed, IFNULL(1.* p1_wins_by_cards/TimesPlayed,0) AS P1ProbWinCard
FROM(
    SELECT p1_seq, p2_seq, p1_wins_by_cards, Count(*) AS TimesPlayed  
    FROM ----
    LEFT JOIN (
        SELECT p1_seq, p2_seq, win_cards, Count(*) as p1_wins_by_cards
        -- records the amount of times for each combination of guesses how many times p1 won by card
        from ----
        WHERE win_cards='p1'
        GROUP BY p1_guess,p2_guess
    ) USING (p1_guess,p2_guess)
    GROUP BY p1_guess, p2_guess
)  
"""
db.run_query(sql1)

In [49]:
db.close()